<a href="https://colab.research.google.com/github/HSE-LAMBDA/MLDM-2021/blob/master/08-neuralnets/MLDM_2021_seminar08_Introduction_to_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brief intro to TensorFlow

![tf logo](https://www.tensorflow.org/images/tf_logo_social.png)

TensorFlow (tf) is one of the popular symbolic math libraries. Roughly, you can think of it as (a subset of) `numpy` with *GPU support* and *automatic differentiation* + a set of useful tools to build and train neural nets.

The computations in tf are represented in the form of computation graph - a directed graph whose edges are the *numerical values* (or 'tensors') to make calculations with, and whose vertices are operations on those values. The edge directions indicate the direction of the computation flow, i.e. what tensors are the inputs and what are the outputs to a given operation (vertex).

Prior to the release of version 2.0, the main execution paradigm in tf was the *lazy evaluation*. This is an evaluation strategy that delays the execution of an operation until its result is actually needed. This means that in order to make computations you need to first define the computation graph and then execute it separately. While such an approach is reasonable for most NNs (you define the network architecture only once and then repeatedly feed the data to it) it might seem a little unintuitive. Also, this paradigm makes debugging quite troublesome. 

https://www.tensorflow.org/guide/

Starting from version 2.0, the default paradigm in tf is the *eager evaluation*, which runs the operations as soon as you create them. This approach might feel more natural and therefore make the development process smoother. It is still possible to use the lazy execution paradigm though.

Let's start this introduction with an overview of the low-level API.

## Low-level intro

In [ ]:
# this line ensures you use the new version of tensorflow:
#%tensorflow_version 2.x

import tensorflow as tf
from IPython.display import clear_output

print(tf.__version__)

# tf.debugging.set_log_device_placement(True)
        # uncommenting the previous line will let you peek on what's
        # happening behind the scenes (i.e. what operations are
        # created, which devices they are using, etc.)

### Basic stuff

Firstly, consider the following code in `numpy`:

In [ ]:
import numpy as np

x = np.array([1., 2., 3.])
y = np.array([3., 2., 1.])

print('---')
print(x + y)

print('---')
print(x * y)

print('---')
print(x.dot(y))

print('---')
print(x[:,np.newaxis].dot(y[np.newaxis,:]))

Here's the equivalent in TensorFlow:

In [ ]:
x = tf.convert_to_tensor([1., 2., 3.])
y = tf.convert_to_tensor([3., 2., 1.])

print('---')
print(x + y)

print('---')
print(x * y)

print('---')
print(tf.matmul(x[tf.newaxis,:], y[:,tf.newaxis]))

print('---')
print(tf.matmul(x[:,tf.newaxis], y[tf.newaxis,:]))

Like `numpy`, TensorFlow is capable of broadcasting unit dimentions, e.g. below we calculate a difference between two tensors of shapes `(3, 1)` and `(1, 3)` to get the result of shape `(3, 3)`:

In [ ]:
z = x[:,tf.newaxis] - y[tf.newaxis,:]
print(z)

Actions like `mean`, `max`, `min`, etc. are implemented with the `reduce_*` opetations, e.g.:

In [ ]:
print(tf.reduce_sum(z, axis=0))

In [ ]:
# now try printing out the max of z along axis 1
<YOUR CODE>

Contrary to `numpy`, you cannot assign to tensors:

In [ ]:
try:
  z[1,1] = 3.
except Exception as e:
  print(e)

Tensors are immutable. If you wish to have a modifiable tensor, then you want to use the `tf.Variable`:

In [ ]:
z_var = tf.Variable(z)
print(z_var)
z_var[1,1].assign(3.)
print(z_var)

In [ ]:
# Exercise (1 point):
# 1) create a normal random vector (tensorflow tensor) of size 5
# 2) create a uniform random vector (tensorflow tensor) of size 5
# 3) get a vector (of size 5) of max values between 1) and 2)
<YOUR CODE>


-----------

### CPU vs GPU

You can check which device's memory an object is stored in:

In [ ]:
x = tf.convert_to_tensor([1., 2., 3.])
y = tf.convert_to_tensor([3., 2., 1.])
z = x[:,tf.newaxis] - y[tf.newaxis,:]

print(x.device)
print(y.device)
print(z.device)

In order to check the available devices you can do:

In [ ]:
quote = "'"
for d in tf.config.list_logical_devices():
  print(f"Name: {quote + d.name + quote:20s}\t Type:'{d.device_type}'")

You can explicitly set the device you want to use for a tensor/operation using `tf.devcie(...)` in a python `with` statement:

In [ ]:
with tf.device('/device:CPU:0'):
  x_cpu = tf.identity(x)
print(x.device)
print(x_cpu.device)

Let's compare the performance of CPU and GPU with calculating a tensor product of two relatively large vectors. First on CPU:

In [ ]:
# We'll leave the object creation outside the timimg part
with tf.device('/device:CPU:0'):
  x = tf.convert_to_tensor(np.arange(10000).astype('float'))

In [ ]:
%%timeit -r1 -n1
with tf.device('/device:CPU:0'):
  print(tf.reduce_sum(tf.matmul(x[:,tf.newaxis], x[tf.newaxis,:])))

Now same code on GPU:

In [ ]:
with tf.device('/device:GPU:0'):
  x = tf.convert_to_tensor(np.arange(10000).astype('float'))

In [ ]:
%%timeit -r1 -n1
with tf.device('/device:GPU:0'):
  print(tf.reduce_sum(tf.matmul(x[:,tf.newaxis], x[tf.newaxis,:])))

Note that TensorFlow is smart enought to place tensors and operations on the most efficient devices automatically. E.g. if we create the object on CPU, but then don't specify the device to run computations on, we'll get the following:

In [ ]:
with tf.device('/device:CPU:0'):
  x = tf.convert_to_tensor(np.arange(10000).astype('float'))

In [ ]:
%%timeit -r1 -n1
print(tf.reduce_sum(tf.matmul(x[:,tf.newaxis], x[tf.newaxis,:])))

----------

### Automatic differentiation

As mentioned earlier, tf can [calculate the derivatives](https://cs231n.github.io/optimization-2/) for you. This can be done with a `tf.GradientTape` object. This object records the operations of interest to later compute the gradients for:

In [ ]:
x = tf.convert_to_tensor(np.linspace(-3, 3, 200))

# The operations we need the gradients for should be enclosed
# in a `with` statement with `tf.GradientTape`:
with tf.GradientTape() as t:
  # specify the tensor we'll derive with respect to
  t.watch(x)

  # do the actual operation:
  y = x**2

# get the gradient:
y_prime = t.gradient(y, x)

import matplotlib.pyplot as plt

plt.plot(x, y, label='y=x^2')
plt.plot(x, y_prime, label='deriv')
plt.legend();

You can nest `tf.GradientTape` contexts to calculate higher order derivatives. Can you figure out the right way to code it? **(2 points)**

In [ ]:
x = tf.convert_to_tensor(np.linspace(-3, 3, 200))

<YOUR CODE>
# y = x**3
# y_prime - first derivative of y wrt x
# y_prime_prime - second derivative of y wrt x

plt.plot(x, y, label='y=x^3')
plt.plot(x, y_prime, label='deriv')
plt.plot(x, y_prime_prime, label='second deriv')
plt.legend();

------------

### Example: single hidden layer NN

For this example we'll try to classify some clothes pictures from the Fashion MNIST dataset. We'll use `tensorflow_datasets` to fetch the data.

In [ ]:
import tensorflow_datasets as tfds

`tf.data.Dataset` is TensorFlow's abstraction for an arbitrary dataset. It allows building complex loading and processing pipelines by sequentially calling dataset transforming functions.

In the code below `tfds.load(...)` returns a `tf.data.Dataset` object, from which we want to load everything at once (`prefetch(number_of_elements)`) and store it in memory (`cache()`). These transformations will be exectuted once we start iterating through the data.

In [ ]:
data_train = tfds.load(name="fashion_mnist", split="train").prefetch(60000).cache()
data_test  = tfds.load(name="fashion_mnist", split="test" ).prefetch(10000).cache()

Let's have a look at the data format:

In [ ]:
print(data_train)
print(data_test)

So we have images of shape 28 by 28 with a single channel and scalar labels.

Now let's just plot some of the training samples.

In [ ]:
# Array for decoding the categories
label_names = np.array(['T-shirt/top',
                        'Trouser',
                        'Pullover',
                        'Dress',
                        'Coat',
                        'Sandal',
                        'Shirt',
                        'Sneaker',
                        'Bag',
                        'Ankle boot'])

# Get a single data batch of 25 images
sample_data = next(iter(data_train.batch(25)))
sample_images = sample_data['image']
sample_labels = sample_data['label']

# Plot the images in a 5x5 grid
plt.figure(figsize=(5, 5))
plt.imshow(
    sample_images.numpy().reshape(5, 5, 28, 28).transpose((0, 2, 1, 3)).reshape(140, 140),
    cmap='gray'
)
# Print corresponding labels
print(label_names[sample_labels.numpy().reshape(5, 5)])

Now that we have some idea about the dataset, we are going to define our model. Since we are working with the low-level API, we are going to define all the calculations explicitly. Therefore, we need to explicitly define all the parameters of our model:

In [ ]:
num_hidden = 64

# Weights and biases of our model.
# We'll use He initialization for weights and zero-initialized biases.
w1 = tf.Variable(
    np.random.normal(size=(28**2, num_hidden)) * np.sqrt(2. / 28**2),
    dtype='float32'
)
b1 = tf.Variable(np.zeros(shape=(num_hidden,)), dtype='float32')

w2 = tf.Variable(
    np.random.normal(size=(num_hidden, 10)) * np.sqrt(2. / num_hidden),
    dtype='float32'
)
b2 = tf.Variable(np.zeros(shape=(10,)), dtype='float32')

variables = [w1, b1, w2, b2]

Now the model itself **(2 points)**:

In [ ]:
# Main function of our model.
# Given the input images it will return the predicted class probabilites
def model(X):
  # preprocess the data first
  X = tf.reshape((X / 255), (X.shape[0], 28**2))

  # First layer
  activations1 = <YOUR CODE> # use tf.nn.relu activation

  # Second layer
  probs = <YOUR CODE> # use tf.nn.softmax activation
  return probs

# The forward pass: calculate the loss value from a given data batch.
def forward(input_batch):
  # preprocessing
  X = input_batch['image']
  y = input_batch['label']
  y = tf.reshape(y, (y.shape[0], 1))

  probs = model(X)

  # Calculate the loss.
  cross_entropy_loss = <YOUR CODE>
    # Hint: tf.gather_nd(probs, y, batch_dims=1) will collect the probabilities of true classes

  return cross_entropy_loss

Now we'll code the main training loop.

In [ ]:
from tqdm import tqdm # a nice tool to track the progress of a loop
import sys

# In earlier examples we updated the variables manually.
# Actually, a set of ready-to-use tools, called optimizers,
# has already been implemented for us.
# Here we'll use the Adam optimizer which is a clever
# modification of simple stochastic gradient descent.
optimizer = tf.optimizers.Adam()
batch_size = 1024

num_epochs = 20

# variables to keep track of the training history
train_losses = []
test_losses = []

# training loop
for i_epoch in range(num_epochs):
  print("Working on epoch #{}".format(i_epoch))
  sys.stdout.flush() # This is to make sure no output is buffered when tqdm is called


  # training part
  epoch_train_loss = 0
  train_samples = 0
  for batch in tqdm(data_train.shuffle(60000).batch(batch_size)):
    with tf.GradientTape() as t:
      loss = <YOUR CODE>
    num_samples = len(batch['label'])
    epoch_train_loss += loss.numpy() * num_samples
    train_samples += num_samples

    grads = <YOUR CODE>
    optimizer.apply_gradients(zip(grads, variables))
  
  train_losses.append(epoch_train_loss / train_samples)

  # evaluation part
  epoch_test_loss = 0
  test_samples = 0
  for batch in data_test.batch(4096):
    epoch_test_loss += forward(batch).numpy() * len(batch['label'])
    test_samples += len(batch['label'])
  
  test_losses.append(epoch_test_loss / test_samples)

  # printout and plotting part
  clear_output(wait=True)
  print("Epoch #{}, train loss: {}, test loss: {}".format(
      i_epoch, train_losses[-1], test_losses[-1]
  ))
  plt.plot(train_losses, label='train')
  plt.plot(test_losses , label='test')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.legend()
  plt.show()

Let's check the accuracy of our model.

In [ ]:
########################################################
# Extract the data from the dataset into a numpy array
X_batches = []
y_batches = []
for batch in data_test.batch(4096):
  X_batches.append(batch['image'].numpy().astype('float32'))
  y_batches.append(batch['label'].numpy())

X_test = np.concatenate(X_batches)
y_test = np.concatenate(y_batches)

In [ ]:
def check_accuracy(model):
  <YOUR CODE> # print out the models accuracy on X_test, y_test

In [ ]:
check_accuracy(model)

Not too bad. Let's have a look at the wrongly classified objects.

In [ ]:
predictions = model(X_test).numpy().argmax(axis=1)
wrong = predictions != y_test
print("True labels:")
print(label_names[y_test[wrong][:9].reshape(3, 3)])
print("Predicted labels:")
print(label_names[predictions[wrong][:9].reshape(3, 3)])

plt.imshow(
  X_test[wrong][:9].reshape(3, 3, 28, 28).transpose(0, 2, 1, 3).reshape(28*3, 28*3)
);

## High-level API (keras)

### Intro

*Quote from https://www.tensorflow.org/guide/keras (see this page for links to mode detailed tutorials):*

> TensorFlow’s high-level APIs are based on the Keras API standard for defining and training neural networks. Keras enables fast prototyping, state-of-the-art research, and production—all with user-friendly APIs.

Simple feed-forward models with single route from input to output can be built with the `tf.keras.Sequential` model:

In [ ]:
model = tf.keras.Sequential([
  # First layer will do the preprocessing and reshaping of the data (typically
  # you want to do this step beforehand, outside of the model, in order to
  # improve the performance)
  tf.keras.layers.Lambda(
      lambda x: tf.reshape(x / 255., (-1, 28*28)),
      input_shape=(28, 28, 1), name='preprocessing'
  ),

  # Densely connected hidden layer
  tf.keras.layers.Dense(num_hidden, activation='relu'),

  # Output layer (note there's no activation, not even softmax:
  # this layer will output logits, there are pre-implemented
  # loss functions in `keras` to work with such outputs)
  tf.keras.layers.Dense(10)
])

Once your model is built you can print its summary:

In [ ]:
model.summary()

Keras models can be easily saved and restored. Here's how to save it to hdf5 format:

In [ ]:
model.save('untrained_model.h5', save_format='h5')

Keras models can be used as functions along with the low-level TensorFlow API, such that you can build a fine-tuned training procedure for your needs. However, most of the supervised learning processes fall into the same routine that has been conveniently pre-implemented in `keras`. In order to use it you want to "compile" your model. That is, assign it an optimizer, a loss function and optionally some validation metrics:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

Once the model is compiled, you can run the training loop using the `fit` method (alternatively, `fit_generator`, see the docs for more info), which can work with different data formats, including tensorflow datasets.

Since our dataset is structured (it has inputs and targets labeled with the 'image' and 'label' keys), we want to convert it to a dataset of tuples `(input, target)`. We'll do that with the `tf.data.Dataset.map` method and the following convertion function:

In [ ]:
def unwrap(x):
  return (x['image'], x['label'])

And finally run the training procedure:

In [ ]:
model.fit(x=data_train.map(unwrap).shuffle(60000).batch(batch_size),
          epochs=10,
          validation_data=data_test.map(unwrap).batch(4096))

Let's save the trained model:

In [ ]:
model.save('trained_model.h5', save_format='h5')

We can now check the accuracy of the model. Let's compare it before and after the training by loading the saved weights:

In [ ]:
model.load_weights('untrained_model.h5')
check_accuracy(model)

In [ ]:
model.load_weights('trained_model.h5')
check_accuracy(model)

Note that we were saving not just the weights, but the entire model (with the computation graph). This means we can load the entire model without having to specify it's architecture:

In [ ]:
check_accuracy(tf.keras.models.load_model('trained_model.h5'))

### Validation metrics and callbacks

Now, let's get back to the untrained state of our model and add some more useful tools to the training process. Earlier we mentined that you can add validation metrics to you model. Here's how its done:

In [ ]:
model.load_weights('untrained_model.h5')
model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])

Another useful thing is keras callbacks - these are function objects to be called at specific events (e.g. at end of batch or epoch processing, etc.).

One of such callbacks can automatically save our model:

In [ ]:
# This callback will save the model every epoch:
saver1 = tf.keras.callbacks.ModelCheckpoint(filepath='model_epoch_{epoch:02d}.h5',
                                            save_freq='epoch')

# This one will save only the best model based on the validation metric (accuracy):
saver2 = tf.keras.callbacks.ModelCheckpoint(filepath='model_best.h5',
                                            monitor='val_sparse_categorical_accuracy',
                                            save_best_only=True)

# We need to provide these callbacks to the `callbacks` argument of the `fit` method:
model.fit(x=data_train.map(unwrap).shuffle(60000).batch(batch_size),
          epochs=30,
          validation_data=data_test.map(unwrap).batch(4096),
          callbacks=[saver1, saver2])

In [ ]:
loaded_model = tf.keras.models.load_model('model_best.h5')
predictions = loaded_model(X_test).numpy().argmax(axis=1)
print("Test accuracy is:", (predictions == y_test).mean())

### Training history

After the `fit` method was called, a history object is attached to the model. This can be used to check how various metrics behaved during the training process:

In [ ]:
plt.plot(model.history.history['loss']    , label='train_loss')
plt.plot(model.history.history['val_loss'], label='test_loss')

plt.plot(model.history.history['sparse_categorical_accuracy'    ], label='train accuracy')
plt.plot(model.history.history['val_sparse_categorical_accuracy'], label='test accuracy')
plt.xlabel('epoch')
plt.ylabel('cross entropy / test accuracy')
plt.legend();

## Miscellaneous: hidden layer activations / ideal ankle boot

Now let's have a look at what features are being extracted by the hidden layer of the best model.

In [ ]:
model = tf.keras.models.load_model('model_best.h5')

In [ ]:
plt.figure(figsize=(10, 10))
hidden_weights = model.layers[1].weights[0].numpy()
hidden_weights /= (hidden_weights**2).sum(axis=0)**0.5

plt.imshow(hidden_weights.T.reshape(8, 8, 28, 28).transpose(0, 2, 1, 3).reshape(28*8, 28*8), cmap='gray');

Here we create a random image and then optimize it to maximize the prediction for a given category:

In [ ]:
import time

In [ ]:
img = tf.Variable(np.random.normal(size=(1, 28*28)), dtype='float32')
category = 9

# Category legend:
# 0 - 'T-shirt/top',
# 1 - 'Trouser',
# 2 - 'Pullover',
# 3 - 'Dress',
# 4 - 'Coat',
# 5 - 'Sandal',
# 6 - 'Shirt',
# 7 - 'Sneaker',
# 8 - 'Bag',
# 9 - 'Ankle boot'

for i in range(1000):
  with tf.GradientTape() as t:
    predictions = model.layers[2](model.layers[1](img))
    score = tf.reduce_min(predictions[:,category:category+1] - 
                          tf.concat([predictions[:,:category],
                                     predictions[:,category+1:]], axis=1),
                          axis=1)

  grads = t.gradient(score, img)
  img.assign_add(0.01 * grads)
  img.assign(img - tf.reduce_min(img, keepdims=True))
  img.assign(img / tf.reduce_max(img))

  if i % 100 == 0:
    clear_output(wait=True)
    plt.imshow(img.numpy().reshape(28, 28), cmap='gray');
    plt.show();
    time.sleep(0.2)
  

In [ ]:
img = (255 * img.numpy())

model(img.reshape(1, 28, 28))